# Topic model

In [13]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

In [15]:
# Data preprocessing
df = pd.read_excel('all_speech.xlsx')

child = df[df.agecat == 'child']
adult = df[df.agecat == 'adult']

data = child.speech.values.tolist() # or: adult
pprint(data[:3])

       Unnamed: 0            speaker agecat  \
40             40  capiteinlatechild  child   
57             57            meisjes  child   
70             70          capitein8  child   
71             71          capitein8  child   
75             75          capitein8  child   
79             79         capitein10  child   
396           396            jongens  child   
397           397             jongen  child   
400           400             jongen  child   
403           403             jongen  child   
404           404             jongen  child   
1255         1255           jonathan  child   
1256         1256           jonathan  child   
1257         1257           jonathan  child   
1258         1258           jonathan  child   
1259         1259           jonathan  child   
1260         1260           jonathan  child   
1261         1261           jonathan  child   
1262         1262           jonathan  child   
1263         1263           jonathan  child   
1264         

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuation

data_words = list(sent_to_words(data))

print(data_words[:3])

[['wat', 'is', 'er', 'wat', 'doen', 'ze'], ['als', 'je', 'je', 'moeder', 'aan', 'het', 'huilen', 'brengt', 'zo', 'erg', 'dat', 'haar', 'tranen', 'op', 'haar', 'borst', 'vallen', 'zou', 'het', 'niet', 'vreemd', 'zijn', 'als', 'je', 'ziek', 'wordt', 'en', 'sterft'], ['agya', 'ee']]


In [20]:
# Functions for stopwords and bigrams
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [21]:
# Remove stop words
data_words_nostops = remove_stopwords(data_words)

# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)
print(id2word)

# Create Corpus
texts = data_words_nostops # or: data_words_bigrams

Dictionary(4509 unique tokens: ['borst', 'brengt', 'erg', 'huilen', 'moeder']...)


In [24]:
# Term frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:5])

[[], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1)], [(12, 1)], [(10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1)]]


In [26]:
# Human readable format of term-frequency
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[],
 [('borst', 1),
  ('brengt', 1),
  ('erg', 1),
  ('huilen', 1),
  ('moeder', 1),
  ('sterft', 1),
  ('tranen', 1),
  ('vallen', 1),
  ('vreemd', 1),
  ('ziek', 1)],
 [('agya', 1), ('ee', 1)],
 [('vader', 1)],
 [('agya', 1),
  ('ee', 1),
  ('vader', 1),
  ('nee', 1),
  ('nooit', 2),
  ('oboroni', 1)],
 [('gebleven', 1), ('mensen', 1), ('waar', 1)],
 [('kapitaan', 1), ('later', 2), ('osofo', 1)],
 [('efua', 1), ('halen', 1), ('manu', 1), ('zoekt', 1)],
 [('wilt', 1), ('wonen', 1)],
 []]

In [27]:
# LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"niks" + 0.041*"vroeg" + 0.031*"ga" + 0.028*"nee" + 0.024*"hoor" + '
  '0.022*"waarom" + 0.018*"vader" + 0.018*"net" + 0.017*"riep" + 0.017*"zag"'),
 (1,
  '0.082*"we" + 0.029*"nou" + 0.028*"waar" + 0.027*"goed" + 0.026*"weer" + '
  '0.018*"wist" + 0.014*"kom" + 0.013*"heen" + 0.013*"weg" + 0.012*"zit"'),
 (2,
  '0.069*"meester" + 0.037*"moeder" + 0.032*"zeg" + 0.029*"weet" + '
  '0.019*"mamma" + 0.016*"ie" + 0.014*"zeggen" + 0.014*"gaat" + 0.014*"ging" + '
  '0.013*"elkaar"'),
 (3,
  '0.051*"zegt" + 0.042*"wel" + 0.027*"caro" + 0.024*"jij" + 0.022*"mimoen" + '
  '0.019*"ogen" + 0.015*"misschien" + 0.014*"terug" + 0.013*"moeten" + '
  '0.013*"mag"'),
 (4,
  '0.109*"zei" + 0.045*"spiek" + 0.034*"opa" + 0.034*"oma" + 0.023*"keek" + '
  '0.021*"gaan" + 0.019*"polleke" + 0.014*"nooit" + 0.012*"huis" + '
  '0.011*"kijkt"')]


In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/envs/clean36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.194894 -0.213621       1        1  22.318140
4     -0.183484  0.272876       2        1  21.333397
2      0.014097 -0.114678       3        1  20.134764
1      0.192075  0.025550       4        1  18.590187
0      0.172205  0.029873       5        1  17.623514, topic_info=     Category        Freq      Term       Total  loglift  logprob
498   Default  741.000000       zei  741.000000  30.0000  30.0000
343   Default  485.000000        we  485.000000  29.0000  29.0000
1325  Default  446.000000   meester  446.000000  28.0000  28.0000
194   Default  367.000000      zegt  367.000000  27.0000  27.0000
87    Default  268.000000      niks  268.000000  26.0000  26.0000
69    Default  297.000000       wel  297.000000  25.0000  25.0000
1961  Default  324.000000     spiek  324.000000  24.0000  24.0000
43    Default  234.000000     vroeg  234.000000  23.0000  23.0000
4     Default  236.000000    moeder  236.000000  22.0000  22.0000
528   Default  231.000000       opa  231.000000  21.0000  21.0000
527   Default  230.000000       oma  230.000000  20.0000  20.0000
135   Default  207.000000       zeg  207.000000  19.0000  19.0000
157   Default  176.000000        ga  176.000000  18.0000  18.0000
151   Default  183.000000      weet  183.000000  17.0000  17.0000
74    Default  170.000000       nou  170.000000  16.0000  16.0000
18    Default  168.000000      waar  168.000000  15.0000  15.0000
13    Default  156.000000       nee  156.000000  14.0000  14.0000
182   Default  160.000000      goed  160.000000  13.0000  13.0000
245   Default  153.000000      weer  153.000000  12.0000  12.0000
51    Default  170.000000       jij  170.000000  11.0000  11.0000
1988  Default  159.000000      keek  159.000000  10.0000  10.0000
142   Default  136.000000      hoor  136.000000   9.0000   9.0000
1922  Default  257.000000      caro  257.000000   8.0000   8.0000
212   Default  142.000000      gaan  142.000000   7.0000   7.0000
231   Default  122.000000    waarom  122.000000   6.0000   6.0000
1937  Default  132.000000   polleke  132.000000   5.0000   5.0000
93    Default  135.000000      ogen  135.000000   4.0000   4.0000
289   Default  122.000000     mamma  122.000000   3.0000   3.0000
1933  Default  219.000000    mimoen  219.000000   2.0000   2.0000
202   Default  106.000000      wist  106.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
43     Topic5  233.704941     vroeg  234.312363   1.7333  -3.1822
157    Topic5  175.987610        ga  176.579956   1.7326  -3.4659
13     Topic5  155.858383       nee  156.442627   1.7322  -3.5874
142    Topic5  136.234512      hoor  136.822540   1.7316  -3.7219
231    Topic5  122.212219    waarom  122.803566   1.7311  -3.8305
12     Topic5  102.626198     vader  103.225578   1.7301  -4.0052
297    Topic5   98.751060       net   99.344124   1.7299  -4.0437
168    Topic5   93.957008  helemaal   94.553291   1.7296  -4.0935
992    Topic5   95.425995       zag   96.034554   1.7296  -4.0779
2177   Topic5   96.412376      riep   97.030411   1.7295  -4.0677
233    Topic5   77.304070    alleen   77.896011   1.7283  -4.2886
277    Topic5   73.280090    gewoon   73.867455   1.7280  -4.3420
1072   Topic5   73.186325    meteen   73.787857   1.7278  -4.3433
226    Topic5   73.430779    straat   74.042442   1.7276  -4.3400
41     Topic5   63.919399      laat   64.510643   1.7267  -4.4787
1235   Topic5   69.251541    binnen   69.900131   1.7266  -4.3986
84     Topic5   59.123726     bijna   59.729511   1.7257  -4.5567
255    Topic5   52.138168     weten   52.741940   1.7244  -4.6824
139    Topic5   50.375404     thuis   50.987293   1.7239  -4.7168
152    Topic5   47.501434      best   48.097572   1.7235  -4.7755
143    Topic5   46.931992      mooi   47.522785   1.7234  -4.7876
1994   Topic5   47.246273      mond   47.850674   1.7232  -4.7809
73     Topic5  